In [1]:
import os
import csv
import numpy as np
import SimpleITK as sitk

In [2]:
# =================================================================
# 1. CONFIGURATION DES CHEMINS
# =================================================================
ROOT_TRAIN = "../../../27919209/MSLesSegDataset/train"
GLOBAL_OUTPUT_DIR = "Moments_Extraction_3D_Results"
MODALITIES = ["T1", "T2", "FLAIR"]
ORDRE_CIBLE = 10 # 10x10x10 = 1000 coefficients

In [3]:
# =================================================================
# 2. MOTEUR MATHÉMATIQUE 3D STABLE
# =================================================================

def tchebichef_polynomials(N, order):
    """Calcule les polynômes via la récurrence de Mukundan."""
    M = np.zeros((order, N))
    x = np.arange(N)
    M[0, :] = 1.0
    if order > 1:
        M[1, :] = (2.0 * x - N + 1.0) / N
    for p in range(2, order):
        term1 = ((2.0 * p - 1.0) * (2.0 * x - N + 1.0)) / (p * N)
        term2 = ((p - 1.0) / p) * (1.0 - ((p - 1.0)**2 / (N**2)))
        M[p, :] = term1 * M[p-1, :] - term2 * M[p-2, :]
    return M

def calculate_rho(N, order):
    """Constante de normalisation pour l'orthogonalité."""
    rho = np.zeros(order)
    rho[0] = N
    for p in range(1, order):
        term1 = (N**2 - p**2) / (N**2)
        term2 = (2*p - 1) / (2*p + 1)
        rho[p] = term1 * term2 * rho[p-1]
    return rho

def decompose_3d(volume, requested_order):
    """Génère le tenseur des moments T_pqr (10x10x10)."""
    D, H, W = volume.shape
    
    # Sécurité pour les dimensions plus petites que l'ordre
    o_d, o_h, o_w = min(requested_order, D), min(requested_order, H), min(requested_order, W)
    
    # Génération des matrices de polynômes
    M_d = tchebichef_polynomials(D, o_d)
    M_h = tchebichef_polynomials(H, o_h)
    M_w = tchebichef_polynomials(W, o_w)
    
    # Normalisation rho
    rho_d = calculate_rho(D, o_d)
    rho_h = calculate_rho(H, o_h)
    rho_w = calculate_rho(W, o_w)
    
    # Calcul des moments par produit tensoriel (np.einsum est le plus efficace ici)
    # T_pqr = sum_z sum_y sum_x f(z,y,x) * Mz(p,z) * My(q,y) * Mx(r,x)
    T_raw = np.einsum('zyx,pz,qy,rx->pqr', volume, M_d, M_h, M_w)
    
    # Application de la normalisation 3D
    norm_factor = 1.0 / (np.einsum('p,q,r->pqr', rho_d, rho_h, rho_w))
    T_pqr_small = T_raw * norm_factor
    
    # Remplissage du tenseur 10x10x10 final
    T_final = np.zeros((requested_order, requested_order, requested_order))
    T_final[:o_d, :o_h, :o_w] = T_pqr_small
    
    # Arrondi à 3 décimales
    return np.round(T_final, 3)

In [4]:
# =================================================================
# 3. BOUCLE MASTER (TOUS PATIENTS, TIMEPOINTS, MODALITÉS)
# =================================================================

os.makedirs(GLOBAL_OUTPUT_DIR, exist_ok=True)
patients = sorted([d for d in os.listdir(ROOT_TRAIN) if os.path.isdir(os.path.join(ROOT_TRAIN, d))])

for patient in patients:
    patient_path = os.path.join(ROOT_TRAIN, patient)
    timepoints = sorted([d for d in os.listdir(patient_path) if os.path.isdir(os.path.join(patient_path, d))])
    
    for tp in timepoints:
        tp_path = os.path.join(patient_path, tp)
        out_dir = os.path.join(GLOBAL_OUTPUT_DIR, patient, tp)
        os.makedirs(out_dir, exist_ok=True)
        
        # 1. Gestion du Masque 3D
        mask_path = os.path.join(tp_path, f"{patient}_{tp}_MASK.nii.gz")
        if not os.path.exists(mask_path): continue
        
        mask_sitk = sitk.ReadImage(mask_path)
        mask_bin = sitk.Cast(mask_sitk > 0, sitk.sitkUInt8)
        # Isolation et tri des lésions par volume
        lesion_cc = sitk.RelabelComponent(sitk.ConnectedComponent(mask_bin))
        mask_arr = sitk.GetArrayFromImage(lesion_cc)
        num_lesions = int(mask_arr.max())
        
        # 2. Boucle sur les modalités
        for modality in MODALITIES:
            img_path = os.path.join(tp_path, f"{patient}_{tp}_{modality}.nii.gz")
            if not os.path.exists(img_path): continue
            
            img_arr = sitk.GetArrayFromImage(sitk.ReadImage(img_path))
            print(f"Traitement 3D : {patient} | {tp} | {modality} ({num_lesions} lésions)")
            
            for l_id in range(1, num_lesions + 1):
                # Extraction du volume de la lésion (ROI 3D)
                indices = np.where(mask_arr == l_id)
                if indices[0].size < 8: continue # Filtre mini (2x2x2 voxels)
                
                # Bounding Box 3D
                z_min, z_max = indices[0].min(), indices[0].max()
                y_min, y_max = indices[1].min(), indices[1].max()
                x_min, x_max = indices[2].min(), indices[2].max()
                
                roi_vol = img_arr[z_min:z_max+1, y_min:y_max+1, x_min:x_max+1]
                roi_mask = (mask_arr[z_min:z_max+1, y_min:y_max+1, x_min:x_max+1] == l_id)
                
                # Signal pur (Intensité * Masque)
                roi_signal = roi_vol * roi_mask
                
                # Normalisation intensité (0-1)
                v_max = np.max(roi_signal)
                roi_norm = roi_signal / v_max if v_max > 0 else roi_signal
                
                # Décomposition 3D
                moments_3d = decompose_3d(roi_norm, ORDRE_CIBLE)
                
                # Sauvegarde : On aplatit les 1000 coefficients en un seul vecteur pour le CSV
                flat_moments = moments_3d.flatten()
                csv_name = f"{patient}_{tp}_{modality}_L{l_id}_Cheby3D.csv"
                
                with open(os.path.join(out_dir, csv_name), "w", newline="") as f:
                    writer = csv.writer(f)
                    # On écrit les 1000 coefficients sur une seule ligne (ou colonne)
                    writer.writerow(flat_moments)

print("\n[TERMINÉ] Extraction 3D de tous les patients effectuée.")

Traitement 3D : P1 | T1 | T1 (18 lésions)
Traitement 3D : P1 | T1 | T2 (18 lésions)
Traitement 3D : P1 | T1 | FLAIR (18 lésions)
Traitement 3D : P1 | T2 | T1 (18 lésions)
Traitement 3D : P1 | T2 | T2 (18 lésions)
Traitement 3D : P1 | T2 | FLAIR (18 lésions)
Traitement 3D : P1 | T3 | T1 (14 lésions)
Traitement 3D : P1 | T3 | T2 (14 lésions)
Traitement 3D : P1 | T3 | FLAIR (14 lésions)
Traitement 3D : P10 | T1 | T1 (35 lésions)
Traitement 3D : P10 | T1 | T2 (35 lésions)
Traitement 3D : P10 | T1 | FLAIR (35 lésions)
Traitement 3D : P10 | T2 | T1 (32 lésions)
Traitement 3D : P10 | T2 | T2 (32 lésions)
Traitement 3D : P10 | T2 | FLAIR (32 lésions)
Traitement 3D : P11 | T1 | T1 (15 lésions)
Traitement 3D : P11 | T1 | T2 (15 lésions)
Traitement 3D : P11 | T1 | FLAIR (15 lésions)
Traitement 3D : P11 | T2 | T1 (10 lésions)
Traitement 3D : P11 | T2 | T2 (10 lésions)
Traitement 3D : P11 | T2 | FLAIR (10 lésions)
Traitement 3D : P12 | T1 | T1 (36 lésions)
Traitement 3D : P12 | T1 | T2 (36 lésions)